In [1]:
import pandas as pd
import numpy as np
import json
import os
import logging
import sys

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s', stream=sys.stdout)

In [1]:
# --- Configuration for Week 8 (Aggregating Round 7 Data) ---
# Previous master file (contains data up to Round 6)
MASTER_FILE_PATH_OLD = 'bbo_master_w07.csv'
# New master file for Week 08 training (will contain data up to Round 7)
MASTER_FILE_PATH_NEW = 'bbo_master_w08.csv'
ADD_DATA_DIR = 'add_data'

# Files for the new data point (Round 7 data)
INPUTS_FILE = 'week07_clean_inputs.json'  # Inputs from your Week 7 submission
OUTPUTS_FILE = 'week07_clean_outputs.json' # Outputs received for your Week 7 inputs

NUM_FUNCTIONS = 8
# The next round of data to be appended is Round 7.
CURRENT_ROUND = 7

# --- CRITICAL: Function dimensionality mapping ---
FUNCTION_DIMS = {
    1: 2, # F1: 2D
    2: 2, # F2: 2D
    3: 3, # F3: 3D
    4: 4, # F4: 4D
    5: 4, # F5: 4D
    6: 5, # F6: 5D
    7: 6, # F7: 6D
    8: 8  # F8: 8D
}

In [1]:
def load_json_file(file_path):
    """Loads a JSON file or returns None with a warning."""
    full_path = os.path.join(ADD_DATA_DIR, file_path)
    if not os.path.exists(full_path):
        logging.error(f"ERROR: Required file '{full_path}' not found.")
        return None
    try:
        with open(full_path, 'r') as f:
            data = json.load(f)
            return data
    except json.JSONDecodeError:
        logging.error(f"ERROR: Could not decode JSON from '{file_path}'. Check file format (must be valid JSON).")
        return None
    except Exception as e:
        logging.error(f"An unexpected error occurred loading {file_path}: {e}")
        return None

In [1]:
def create_master_data():
    logging.info("*"*50)
    logging.info(f"--- Starting BBO Master File Creation for Round {CURRENT_ROUND} ---")

    # 1. Load the previous master file and apply 'Round' column fix
    if not os.path.exists(MASTER_FILE_PATH_OLD):
        logging.error(f"Master file '{MASTER_FILE_PATH_OLD}' not found. Cannot proceed with aggregation.")
        return None

    try:
        df_master_old = pd.read_csv(MASTER_FILE_PATH_OLD)
        logging.info(f"Loaded {len(df_master_old)} rows from {MASTER_FILE_PATH_OLD}.")

        # FIX: Ensure the 'Round' column exists and is positioned correctly (R0 to R{CURRENT_ROUND - 1})
        num_rounds_after_r0 = (len(df_master_old) - 80) // 8
        round_list = [0] * 80 # Initial 80 rows are Round 0
        for r in range(1, num_rounds_after_r0 + 1):
            round_list.extend([r] * 8)

        if len(round_list) == len(df_master_old):
            if 'Round' in df_master_old.columns:
                df_master_old['Round'] = round_list
            else:
                df_master_old.insert(1, 'Round', round_list)
            logging.info(f"FIX APPLIED: Successfully ensured 'Round' column (R0 to R{num_rounds_after_r0}) is present and correct.")
        else:
            logging.warning(f"Could not reliably infer 'Round' numbers. Skipping round correction.")

    except Exception as e:
        logging.error(f"Failed to read or process master CSV: {e}")
        return None

    # 2. Load the new inputs and outputs
    inputs_list = load_json_file(INPUTS_FILE)
    outputs_array = load_json_file(OUTPUTS_FILE)

    if inputs_list is None or outputs_array is None:
        logging.error("Aggregation aborted due to missing or invalid JSON files.")
        return None

    # --- CRITICAL INPUT VALIDATION STEP ---
    if not isinstance(inputs_list, list) or len(inputs_list) != NUM_FUNCTIONS:
        logging.critical(f"FATAL ERROR: Input file '{INPUTS_FILE}' is invalid or has an incorrect number of entries.")
        logging.critical(f"Expected a list of {NUM_FUNCTIONS} entries (one list of coordinates per function).")
        logging.critical("Aggregation aborted.")
        return None

    if not isinstance(outputs_array, list) or len(outputs_array) != NUM_FUNCTIONS:
        logging.critical(f"FATAL ERROR: Output file '{OUTPUTS_FILE}' is invalid or has an incorrect number of entries.")
        logging.critical("Aggregation aborted.")
        return None

    # Define the columns for the new DataFrame (must match the old one)
    cols_to_use = df_master_old.columns.tolist()

    # 3. Construct the new 8 rows (one for each function)
    new_data = []

    for i in range(NUM_FUNCTIONS):
        f_id = i + 1
        dim = FUNCTION_DIMS[f_id]
        f_name = f'f{f_id}'

        # Safety check for input structure
        x_values_raw = inputs_list[i] if i < len(inputs_list) and isinstance(inputs_list[i], list) else []
        score = outputs_array[i] if i < len(outputs_array) else np.nan

        # Validation and Padding
        if len(x_values_raw) != dim:
            # New logic: automatically truncate the input coordinates to the required dimension
            logging.warning(f"Input dimension mismatch for {f_name}. Expected {dim}, found {len(x_values_raw)}. TRUNCATING inputs to match expected dimension.")

            x_values_to_use = x_values_raw[:dim]

            # If the raw input list was completely empty, we need to use NaNs for the coordinates
            if not x_values_raw:
                logging.warning(f"Raw input list for {f_name} was empty. Using NaNs for coordinates.")
                x_values = [np.nan] * 8
            else:
                # Fill the row with the correct number of NaNs for X-columns beyond 'dim'
                x_values = x_values_to_use + [np.nan] * (8 - dim)
        else:
            # Inputs are perfect, pad with NaNs
            x_values = x_values_raw + [np.nan] * (8 - dim)

        # Create the new row dictionary
        row_data = {
            'Function ID': f_id,
            'Y': score,
            'Round': CURRENT_ROUND
        }

        # Add the X-coordinates
        for d in range(8):
            row_data[f'X{d+1}'] = x_values[d]

        new_data.append(row_data)

    # Create new DataFrame and ensure column order is preserved
    df_new_rows = pd.DataFrame(new_data, columns=cols_to_use)
    logging.info(f"Successfully constructed {len(df_new_rows)} new rows (Round {CURRENT_ROUND}).")

    # 4. Concatenate and Save
    df_master_new = pd.concat([df_master_old, df_new_rows], ignore_index=True)
    df_master_new.to_csv(MASTER_FILE_PATH_NEW, index=False)

    # 5. Verification
    total_rows = len(df_master_new)
    expected_rows = len(df_master_old) + NUM_FUNCTIONS
    logging.info("\n" + "*"*50)
    logging.info(f"SUCCESS: New master data file '{MASTER_FILE_PATH_NEW}' created.")
    # Total rows = 17 initial * 8 functions + 7 rounds * 8 functions = 136 + 56 = 192 rows (24 rows per function)
    logging.info(f"Total rows in the new file: {total_rows} (Expected {expected_rows}).")
    logging.info(f"Verification: There are now {total_rows / 8:.0f} data points (Rounds 0-{CURRENT_ROUND}) for each of the 8 functions.")
    logging.info("*"*50)

    return df_master_new

In [1]:
if __name__ == '__main__':
    if not os.path.exists(ADD_DATA_DIR):
        logging.warning(f"Directory '{ADD_DATA_DIR}' not found. Please create it and place your JSON files inside.")

    df_final = create_master_data()
    if df_final is not None:
        print(f"\n--- Tail of {MASTER_FILE_PATH_NEW} (Showing the 8 newly appended rows) ---")
        # Display the key columns to confirm structure and padding
        print(df_final[['Function ID', 'Round', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'Y']].tail(8))

INFO: **************************************************
INFO: --- Starting BBO Master File Creation for Round 7 ---
INFO: Loaded 128 rows from bbo_master_w07.csv.
INFO: FIX APPLIED: Successfully ensured 'Round' column (R0 to R6) is present and correct.
INFO: Successfully constructed 8 new rows (Round 7).
INFO: 
**************************************************
INFO: SUCCESS: New master data file 'bbo_master_w08.csv' created.
INFO: Total rows in the new file: 136 (Expected 136).
INFO: Verification: There are now 17 data points (Rounds 0-7) for each of the 8 functions.
INFO: **************************************************

--- Tail of bbo_master_w08.csv (Showing the 8 newly appended rows) ---
     Function ID  Round        X1        X2        X3        X4        X5  \
128            1      7  0.190537  0.634466       NaN       NaN       NaN   
129            2      7  0.702802  0.007948       NaN       NaN       NaN   
130            3      7  0.189635  0.000000  0.613410       NaN 